In [4]:
import openai
from dotenv import load_dotenv
import os
import json

In [5]:
load_dotenv(".env")
openai.api_key = os.getenv("OPENAI_API_KEY")

### Job description with image

In [4]:
import openai
from dotenv import load_dotenv
import os
import json

load_dotenv(".env")
openai.api_key = os.getenv("OPENAI_API_KEY")

def create_html_image_tag(job_url_path):
    return f'<img src="{job_url_path}" alt="Job image">'

messages = [
    {"role": "user", "content": "I am a web developer"}
]

job_description = '''Write a detailed description of the job of the user,
assign the job to a category,
and generate an image file path.'''

def get_job_info(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=[
        {
            "name": "job_description_with_image",
            "description": f"{job_description}",
            "parameters": {
                "type": "object",
                "properties": {
                    "job_url_path": {
                        "type": "string", 
                        "enum": ["images/medical", "images/software", "images/farming"],
                        "description": "choose an image path based on the job category",
                    },
                },
            },
        }],
        function_call="auto",
    )
    
    return response

get_job_info(messages)

<OpenAIObject chat.completion id=chatcmpl-7UAMWKi6F0IWLVffOsiqSmaTUUMxY at 0x11fbea4b0> JSON: {
  "id": "chatcmpl-7UAMWKi6F0IWLVffOsiqSmaTUUMxY",
  "object": "chat.completion",
  "created": 1687424028,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "job_description_with_image",
          "arguments": "{\n  \"job_url_path\": \"images/software\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 100,
    "completion_tokens": 20,
    "total_tokens": 120
  }
}

### Full function to generate text plus image

In [25]:
import openai
import json

# Function to create html image tag
def create_html_image_tag(job_url_path):
    return f'<img src="{job_url_path}" alt="Job image">'


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{
        "role": "system", 
        "content": """
        You are a job coach. You write short descriptions to help people find a new career.
        You generate an image url to illustrate the image. \
        To generate the image url you call "create_html_image_tag".
        
        Example:
        Title: <Job title>
        Description: <Short description>
        Image: <img src="{job_url_path}" alt="Job image">
        
        Write a job description for a programmer.
        """
    }]
    functions = [
        {
            "name": "create_html_image_tag",
            "description": "Create HTML image tag for a given URL path",
            "parameters": {
                "type": "object",
                "properties": {
                    "job_url_path": {
                        "type": "string", 
                        "enum": ["images/medical", "images/software", "images/farming"],
                        "description": "choose an image path based on the job category",
                    }
                },
                "required": ["job_url_path"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    print("Response:", response)
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_html_image_tag": create_html_image_tag,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(**function_args)

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


print(run_conversation())


Response: {
  "id": "chatcmpl-7UDapcKR2w35jQIwONcQBdDUd3y1k",
  "object": "chat.completion",
  "created": 1687436447,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Title: Programmer\nDescription: As a programmer, you will be responsible for writing, testing, and maintaining computer software. You will work closely with a team of developers to create high-quality code that meets the needs of the organization or client. A strong understanding of programming languages and problem-solving skills is essential for success in this role. If you enjoy problem-solving and have a passion for technology, a career as a programmer may be a great fit for you.\n\nImage: ![Job image](images/software)",
        "function_call": {
          "name": "create_html_image_tag",
          "arguments": "{\"job_url_path\": \"images/software\"}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usag